In [21]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import calendar

In [22]:
df = pd.read_excel(r'C:\Medline\2. CPM\3. ISP Complaints\ISP Complaints Data 202001-202401.xlsx',sheet_name='ISP Combine')

In [23]:
df = df.loc[df['Year'] >= 2022]

In [24]:
vendor_mapping = pd.read_excel(r'C:\Medline\2. CPM\data\vendor_mapping\Vendor _mapping 2024_v1.xlsx')
vendor_mapping['Vendor Number'] = vendor_mapping['Vendor Number'].map(str)
vendor_mapping_dict = dict(zip(vendor_mapping['Vendor Number'],vendor_mapping['Cleaned Vendor Name']))

In [25]:
df_isp_buom_2022 = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\ISP_ShippingData_2022.xlsx',usecols='A,C,D,AA,BD')
df_isp_buom_2023 = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\ISP_ShippingData_2023.xlsx',usecols='A,C,D,AA,BD')
df_isp_buom_2024 = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\2024\ISP\ISP BUOM 202401.xlsx',usecols='A,B,C,Q,T')
df_isp_buom = pd.concat([df_isp_buom_2024,df_isp_buom_2023,df_isp_buom_2022],ignore_index=True)
df_isp_buom['VendorNumber'] = df_isp_buom['VendorNumber'].map(str)

In [26]:
vendor_mapping_inspection = vendor_mapping.loc[(~vendor_mapping['Regional Manager'].isin(['Exemption','US vendor']))&(vendor_mapping['Regional Manager'].notnull()),'Vendor Number'].to_list()
df_isp_buom = df_isp_buom.loc[df_isp_buom['VendorNumber'].isin(vendor_mapping_inspection)]
df_isp_buom['VendorName'] = df_isp_buom['VendorNumber'].apply(lambda x : vendor_mapping_dict.get(x,np.nan))

In [27]:
df_isp_buom

,SourceERPInd,ProductDivisionCode,VendorNumber,BUOMShipmentQty,ShipmentDate,VendorName
0,QADEU,21,147612,720.0,2024-01-11,LIANYUNGANG ANSON NON WOVEN PRODUC
1,QADEU,21,147612,720.0,2024-01-18,LIANYUNGANG ANSON NON WOVEN PRODUC
2,QADEU,21,147612,1440.0,2024-01-31,LIANYUNGANG ANSON NON WOVEN PRODUC
3,QADEU,15,147612,10000.0,2024-01-11,LIANYUNGANG ANSON NON WOVEN PRODUC
4,QADEU,15,147612,3750.0,2024-01-31,LIANYUNGANG ANSON NON WOVEN PRODUC
...,...,...,...,...,...,...
157479,SAPCA,40,130324,174000.0,2022-10-04,HONGRAY (USA) MEDICAL PRODUCTS INC
157480,SAPCA,40,130324,690000.0,2022-12-10,HONGRAY (USA) MEDICAL PRODUCTS INC
157481,SAPCA,40,130324,690000.0,2022-12-10,HONGRAY (USA) MEDICAL PRODUCTS INC
157482,SAPCA,40,122928,110000.0,2022-07-27,BETTER HEALTH MEDICAL PRODUCTS HONG


In [28]:
df_isp_buom.rename(columns={'SourceERPInd': 'Market',
                            'ProductDivisionCode':'Division',
                            'BUOMShipmentQty':'BUOM'
                            }, inplace=True)
df_isp_buom['Year'] = df_isp_buom['ShipmentDate'].dt.year
df_isp_buom['Month'] = df_isp_buom['ShipmentDate'].dt.month
df_isp_buom['Market'].replace({'QADEU':'EU',
                                'MFGEU':'EU',
                                'QADAU':'ANZ',
                                'QADJP':'JAPAN'},inplace=True)
df_isp_buom = df_isp_buom.loc[df_isp_buom['Market'].isin(['EU','ANZ','JAPAN'])]

In [29]:
df_isp_buom_gy = df_isp_buom.groupby(['Market','VendorName','Division','Year','Month'],as_index=False)['BUOM'].sum()

In [30]:
df_isp_buom_gy[df_isp_buom_gy['BUOM']==0]

,Market,VendorName,Division,Year,Month,BUOM
3818,JAPAN,COBES INDUSTRIES CO LTD,21,2022,9,0.0
3819,JAPAN,COBES INDUSTRIES CO LTD,21,2022,10,0.0
3820,JAPAN,COBES INDUSTRIES CO LTD,21,2022,12,0.0
3821,JAPAN,COBES INDUSTRIES CO LTD,21,2023,4,0.0
3822,JAPAN,COBES INDUSTRIES CO LTD,21,2023,10,0.0
4168,JAPAN,HONGRAY (USA) MEDICAL PRODUCTS INC,21,2022,1,0.0
4221,JAPAN,JIE GAO PLASTICS PRODUCTS CO LTD,15,2022,5,0.0
4222,JAPAN,JIE GAO PLASTICS PRODUCTS CO LTD,15,2023,9,0.0
4752,JAPAN,ZHEJIANG BEAUTY AND HEALTH ELECTRIC,15,2023,5,0.0


In [31]:
df_gy = df.groupby(['Market','Vendor Name','Division','Year','Month'],as_index=False).size()
df_gy.rename(columns={'Vendor Name':'VendorName'},inplace=True)

In [32]:
df_gy[df_gy['size']==0]

,Market,VendorName,Division,Year,Month,size


In [33]:
df_output = df_isp_buom_gy.merge(df_gy,how='outer',on=['Market','VendorName','Division','Year','Month']).fillna(0)

In [34]:
df_output['Year'] = df_output['Year'].astype(str)
df_output['Month'] = df_output['Month'].astype(str)

In [35]:
df_output['Date'] =  pd.to_datetime(df_output['Year'] + df_output['Month'], format='%Y%m')

In [36]:
df_output.rename(columns={'size':'Complaints'},inplace=True)
df_output = df_output[['Market','VendorName','Division','Date','Year','Month','Complaints','BUOM']]

In [38]:
df_output.to_excel('df_output.xlsx',index=False)